# 预处理assets，整理出后续要用的数据

In [1]:
from dotenv import load_dotenv
import os
os.environ['DEBUG'] = '1'
os.environ['SHOW_LLM_INPUT_MSG'] = '1'

# 加载 .env 文件
load_dotenv()

True

In [5]:
import pandas as pd
import json
import llms
from src.utils import show

In [7]:
# Preprocess the competition questions here
root_dir = os.getcwd()
question_data_path = root_dir + '/assets/question.json'
df1 = pd.read_excel(root_dir + '/assets/数据字典.xlsx', sheet_name='库表关系')
df2 = pd.read_excel(root_dir + '/assets/数据字典.xlsx', sheet_name='表字段信息')
file_path = root_dir + '/assets/all_tables_schema.txt'

In [ ]:
# 假设 df1 的列名为 '库名英文', '库名中文', '表英文', '表中文', '表描述'
db_table = df1.groupby('库名英文').apply(lambda x: {
    '库名中文': x['库名中文'].iloc[0],
    '表': x[['表英文', '表中文', '表描述']].apply(lambda y: y.str.lower() if y.name == '表英文' else y).to_dict('records')
}).to_dict()
db_table = {k.lower(): v for k, v in db_table.items()}

# 打印结果以验证
with open(root_dir + '/assets/db_table.json', 'w', encoding='utf-8') as json_file:
    json.dump(db_table, json_file, ensure_ascii=False, indent=2)

/tmp/ipykernel_4099366/2008785078.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  db_table = df1.groupby('库名英文').apply(lambda x: {


In [9]:
# 假设 df2 的列名为 'table_name', 'column_name', 'column_description', '注释'
table_column = df2.groupby('table_name').apply(lambda x: [
    {
        'column': row['column_name'],
        'desc': f"{row['column_description']}" + (f"；{row['注释']}" if pd.notna(row['注释']) and row['注释'] else "").strip()
    }
    for _, row in x.iterrows() if pd.notna(row['column_name']) and row['column_name'] not in ['JSID', 'UpdateTime', 'InsertTime', 'ID', 'XGRQ']
]).to_dict()
table_column = {k.lower(): v for k, v in table_column.items()}

# 打印结果以验证
with open(root_dir + '/assets/table_column.json', 'w', encoding='utf-8') as json_file:
    json.dump(table_column, json_file, ensure_ascii=False, indent=2)

/tmp/ipykernel_4099366/1464820833.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  table_column = df2.groupby('table_name').apply(lambda x: [


In [ ]:
for db_name, db in db_table.items():
    for table in db["表"]:
        table_name = table["表英文"]
        print(f"{db_name}.{table_name}")
        column_list = []
        for column in table_column[table_name]:
            column_name = column["column"]
            column_simple_desc = str(column["desc"]).split("；")[0]
            column_list.append(f"{column_simple_desc}({column_name})")
        all_cols = ",".join(column_list)
        table["all_cols"] = all_cols
        cols_summary, token, ok = llms.llm_gpt_4o.generate_response(
            system='''你善于对数据表的字段信息进行总结，把同类信息归类，比如"联系人电话、联系人传真"等总结为"联系方式如电话、传真等。
        输出一段文字，不换行。"''',
            messages=[
                {
                    "role": "user",
                    "content": f"下面是一个数据表的所有表字段，请帮我为这个数据表写一段介绍，把字段信息压缩进去：\n{all_cols}"
                }
            ],
            stream=False,
        )
        table["cols_summary"] = cols_summary
        if not ok:
            print(f"err: {db_name}.{table_name}")

In [41]:
with open(root_dir + '/assets/db_table.json', 'w', encoding='utf-8') as json_file:
    json.dump(db_table, json_file, ensure_ascii=False, indent=2)

In [ ]:
db_info = []
for db_name, db in db_table.items():
    print(db_name)
    db_json = json.dumps(db, ensure_ascii=False)
    db_summary, token, ok = llms.llm_gpt_4o.generate_response(
    system='''你善于对数据库的表信息进行总结，根据它包含的数据表和字段信息，描述这个数据库，如"本库名为xxx，记录了xxx；涵盖了xxx；方便用户xxx"。
输出一段文字，不换行。"''',
    messages=[
        {
            "role": "user",
            "content": f"下面是一个数据库的所有表和字段信息，请帮我为这个数据库写一段介绍，把表和字段信息压缩进去：\n{db}"
        }
    ],
    stream=False,
)
    db_info.append({
        "db_name": db_name,
        "db_desc": db_summary,
    })

In [48]:
with open(root_dir + '/assets/db_info.json', 'w', encoding='utf-8') as json_file:
    json.dump(db_info, json_file, ensure_ascii=False, indent=2)